# Recommendation System Notebook Sample

0. [Params](#Params)
1. [Acquisitor and Cleaner](#Acquisitor-and-Cleaner)
2. [Training Preparator](#Training-Preparator)
3. [Trainer](#Trainer)
4. [Metrics Evaluator](#Metrics-Evaluator)
5. [Prediction Preparator](#Prediction-Preparator)
6. [Predictor](#Predictor)
7. [Feedback](#Feedback)
8. [Sample Application](#Sample-Application)

# Recommendation System

In [2]:
import marvin_recommendation_system_engine

# Params

In [1]:
params = {
    "param_grid": {
        "n_epochs": [20], 
        "lr_all": [0.005],
        "reg_all": [0.6]  
    },
    "algo": [
        {
            "name": "KNNBaseline",
            "param_grid": {
                'k': [40],
                'sim_options': {
                    'name': ['msd', 'cosine'],
                    'min_support': [5],
                    'user_based': [False]
                }
            }
        },
        {
            "name": "SVD",
            "param_grid": {
                "n_epochs": [10], 
                "lr_all": [0.005],
                "reg_all": [0.6]   
            }
        }
            
            
    ],
    "measures": ["rmse", "mae"],
    "n_cv": 3,
    
    "prediction": {
        "pred_type": "all",
        "n_pred": 10
    }
}
#epochs': [5, 10, 20], 
#'lr_all': [0.002, 0.005],
#'reg_all': [0.4, 0.6]  

# Aquisitor and Cleaner

In [2]:
from surprise import Dataset

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

marvin_initial_dataset = {
    "data": data
}

# Training Preparator

In [3]:
trainset = marvin_initial_dataset["data"].build_full_trainset()
print(trainset.global_mean)
testset = trainset.build_anti_testset()
print(testset[0])
marvin_dataset = {
    "data": marvin_initial_dataset["data"],
    "trainset": trainset,
    "testset": testset
}

3.52986
(u'196', u'302', 3.5298600000000002)


# Trainer

In [4]:
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import KNNBaseline

algo_dict = {"SVD": SVD, "KNNBaseline": KNNBaseline}

model_dict = {}

for algo in params["algo"]:
    
    print(algo)
    
    # Get Name and Initiate Algorithm
    algo_name = algo["name"]
    model_dict[algo_name] = {}
        
    # Initialize Gridsearch
    gs = GridSearchCV(
        algo_dict[algo_name],
        algo["param_grid"],
        measures=params["measures"],
        cv=params["n_cv"])
    
    gs.fit(marvin_dataset["data"])
    
    # We can now use the algorithm that yields the best rmse:
    best_algo = gs.best_estimator['rmse']
    best_algo.fit(marvin_dataset["trainset"])

    # Get the predictions for null values in the set
    model_dict[algo_name]["grid_search"] = gs
    model_dict[algo_name]["model"] = best_algo
    
marvin_model = model_dict

{'param_grid': {'sim_options': {'user_based': [False], 'name': ['msd', 'cosine'], 'min_support': [5]}, 'k': [40]}, 'name': 'KNNBaseline'}
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
{'param_grid': {'lr_all': [0.005], 'reg_all': [0.6], 'n_epochs': [10]}, 'name': 'SVD'}


# Metrics Evaluator

In [ ]:
import pandas as pd
from surprise import accuracy

for algo in params["algo"]:
    print(algo)
    # combination of parameters that gave the best RMSE score
    print("Best Model: {}".format([key + ": " + str(value) for (key,value) in marvin_model[algo["name"]]["grid_search"].best_params['rmse'].items()]))

    # best RMSE score
    print("Best RMSE: {}".format(marvin_model[algo["name"]]["grid_search"].best_score['rmse']))
    
    # Prediction Score
    # Train the algorithm on the trainset, and predict ratings for the testset
    predictions = marvin_model[algo["name"]]["model"].test(marvin_dataset["testset"])

    # Then compute RMSE
    print("Test Set Score: {}".format(accuracy.rmse(predictions)))




Best Model: ["sim_options: {'min_support': 5, 'name': 'msd', 'user_based': False}", 'k: 40']
Best RMSE: 0.939209420679
RMSE: 0.6027
Test Set Score: 0.602708383071
Best Model: ['lr_all: 0.005', 'reg_all: 0.6', 'n_epochs: 10']
Best RMSE: 0.973653369913


In [ ]:
from collections import defaultdict
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

#predictions = marvin_model["model"].test(marvin_dataset["testset"])
#precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

# Precision and recall can then be averaged over all users
#print(sum(prec for prec in precisions.values()) / float(len(precisions)))
#print(sum(rec for rec in recalls.values()) / float(len(recalls)))

# Prediction Preparator

In [ ]:
input_message = {
    "User_id": 196,
    "Item_id": 302
}

In [ ]:
# get a prediction for specific users and items.
pred_dict = {}

for algo in params["algo"]:
    pred_dict[algo["name"]] = marvin_model[algo["name"]]["model"].predict(
        str(input_message["User_id"]), str(input_message["Item_id"]), r_ui=4, verbose=True)
                                           

final_prediction = pred_dict